<a href="https://colab.research.google.com/github/sankalp-omkar/summer-of-code-2024/blob/main/devsoc2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting/data")

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from keras import models
from keras import layers
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error


data = pd.read_csv('//content//sales-forecasting//train.csv')

target = data.Sales

encoder = OrdinalEncoder()

cols_to_encode = ['Ship Mode', 'Segment', 'State', 'Region', 'Category', 'Sub-Category']

data[cols_to_encode] = encoder.fit_transform(data[cols_to_encode])

features=data[cols_to_encode]

def predict_arima():
  model_arima = ARIMA(data, order=(1, 1, 0))
  model_arima_fit = model_arima.fit()
  return model_arima_fit.forecast(steps=1)[0]

def predict_prophet(X_train, y_train):
  model_prophet = Prophet()
  model_prophet.fit(pd.DataFrame({"ds": X_train.index, "y": y_train}))
  future = model_prophet.make_future_dataframe(periods=1)
  forecast_prophet = model_prophet.predict(future)
  return forecast_prophet["sales"].iloc[-1]

def predict_lstm(X_train, y_train):
  model_lstm = models.Sequential()
  model_lstm.add(layers.LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
  model_lstm.add(layers.LSTM(50))
  model_lstm.add(layers.Dense(1))
  model_lstm.compile(loss="mse", optimizer="adam")
  model_lstm.fit(X_train.values.reshape(-1, X_train.shape[1], 1), y_train, epochs=10)
  predicted_value = model_lstm.predict(X_train.values.reshape(-1, X_train.shape[1], 1))[-1, 0]
  return predicted_value

train_size = int(len(data) * 0.8)  # Adjust split ratio as needed
X_train, X_test = features.iloc[:train_size], features.iloc[train_size:]
y_train, y_test = target.iloc[:train_size], target.iloc[train_size:]

arima_pred = predict_arima()
prophet_pred = predict_prophet(X_train, y_train)
lstm_pred = predict_lstm(X_train, y_train)

print("ARIMA Prediction:", arima_pred)
print("Prophet Prediction:", prophet_pred)
print("LSTM Prediction:", lstm_pred)

mse_arima = mean_squared_error(y_test, arima_pred)
mse_prophet = mean_squared_error(y_test, prophet_pred)
mse_lstm = mean_squared_error(y_test, lstm_pred)
print("ARIMA MSE:", mse_arima)
print("Prophet MSE:", mse_prophet)
print("LSTM MSE:", mse_lstm)